# Iterative Closest Point
Resources:  
https://github.com/ClayFlannigan/icp  
https://github.com/agnivsen/icp

In [1]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import time

In [2]:
# Change these two variables to select file path for input and output
# Remember to include "/" at the end of the file_path
file_path = "data-offline/"
csv_name_1 = "1503619123111707211-cloudpoint.csv"
csv_name_2 = "1503619123211929083-cloudpoint.csv"

df_1 = pd.read_csv(file_path + csv_name_1, 
                 names = ['x', 'y', 'z', 
                          'intensity', 'ring', 'rotation', 'revolution'])
df_2 = pd.read_csv(file_path + csv_name_2, 
                 names = ['x', 'y', 'z', 
                          'intensity', 'ring', 'rotation', 'revolution'])

df_1['distance'] = np.sqrt(df_1['x']**2 + df_1['y']**2 + df_1['z']**2)
df_2['distance'] = np.sqrt(df_2['x']**2 + df_2['y']**2 + df_2['z']**2)

df_1 = df_1[df_1.distance > 2.5]
df_2 = df_2[df_2.distance > 2.5]

In [3]:
def best_fit_transform(A, B):
    '''
    Calculates the least-squares best-fit transform that 
    maps corresponding points A to B in m spatial dimensions
    Input:
        A: Nx3 numpy array of corresponding 3D points
        B: Nx3 numpy array of corresponding 3D points
    Returns:
        T: 4x4 homogeneous transformation matrix that maps A on to B
        R: 3x3 rotation matrix
        t: 3x1 translation vector
    '''
    
    assert A.shape[1] == B.shape[1]

    # translate points to their centroids
    centroid_A = np.mean(A, axis=0)
    centroid_B = np.mean(B, axis=0)
    AA = A - centroid_A
    BB = B - centroid_B

    # rotation matrix
    H = np.dot(AA.T, BB)
    U, S, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)

    # special reflection case
    if np.linalg.det(R) < 0:
        Vt[2,:] *= -1
        R = np.dot(Vt.T, U.T)

    # translation
    t = centroid_B.T - np.dot(R,centroid_A.T)

    # homogeneous transformation
    T = np.identity(4)
    T[0:3, 0:3] = R
    T[0:3, 3] = t

    return T, R, t

In [4]:
def nearest_neighbor(src, dest):
    '''
    Find the nearest (Euclidean) neighbor in dest for each point in src
    Input:
        src: Nx3 array of points
        dest: Nx3 array of points
    Output:
        distances: Euclidean distances of the nearest neighbor
        indices: dest indices of the nearest neighbor
    '''
    
    assert src.shape[1] == dest.shape[1]

    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(dest)
    distances, indices = neigh.kneighbors(src, return_distance=True)
    
    return distances.ravel(), indices.ravel()

In [5]:
def point_to_plane_projection(src_point, plane_normal_vec, plane_point):
    v = src_point - plane_point
    dist = np.dot(v, plane_normal_vec)
    point = src_point - dist * plane_normal_vec
    
    return dist, point

In [6]:
def nearest_neighbor_planar(src, dest):
    '''
    For each point in src, find the scalar projection onto
    the planar surface in dest that minimizes Euclidean distance
    Input:
        src: Nx3 array of points
        dest: Nx3 array of points
    Output:
        distances: Euclidean distances to the planes
        points: points in dest projected from src
    '''
    
    assert src.shape[1] == dest.shape[1]

    new_distances = np.zeros((src.shape))
    new_points = np.zeros((src.shape))
    
    neigh = NearestNeighbors(n_neighbors=3)
    neigh.fit(dest)
    distances, indices = neigh.kneighbors(src, return_distance=True)
    
    points = dest[indices,:]

    centroid_points = np.mean(points, axis=1)
    centroid_points = np.reshape(centroid_points, (centroid_points.shape[0], 1, centroid_points.shape[1]))
    matrix = points - centroid_points
    matrix = matrix.transpose((0, 2, 1))
    N = matrix.shape[2]
    m1 = matrix - matrix.sum(2,keepdims=1)/N
    cov_matrices = np.einsum('ijk,ilk->ijl',m1,m1)/(N-1)
    
    for i in range(cov_matrices.shape[0]):
        cov_matrix = cov_matrices[i]
        eigvals, eigvecs = np.linalg.eigh(cov_matrix)
        normal_vec = eigvecs[:,0]

        dist, point = point_to_plane_projection(src[i], normal_vec, centroid_points[i][0])
        new_distances[i] = dist
        new_points[i] = point
    
    return new_distances, new_points

In [7]:
def icp(A, B, init_pose=None, max_iterations=20, threshold=0.001):
    '''
    The Iterative Closest Point method: finds best-fit transform that 
    maps points A on to points B
    Input:
        A: Nx3 numpy array of source 3D points
        B: Nx3 numpy array of destination 3D point
        init_pose: 4x4 homogeneous transformation
        max_iterations: exit algorithm after max_iterations
        threshold: convergence criteria
    Output:
        T: final homogeneous transformation that maps A on to B
        distances: Euclidean distances (errors) of the nearest neighbor
        i: number of iterations to converge
    '''
    
    assert A.shape[1] == B.shape[1]
    
    start_time = time.time()

    # make points homogeneous, copy them so as to maintain the originals
    src = np.ones((4, A.shape[0]))
    dst = np.ones((4, B.shape[0]))
    src[0:3,:] = np.copy(A.T)
    dst[0:3,:] = np.copy(B.T)

    # apply the initial pose estimation
    if init_pose is not None:
        src = np.dot(init_pose, src)

    prev_error = 0

    for i in range(max_iterations):
        # find the nearest neighbours between the current source and destination points
        distances, indices = nearest_neighbor(src[0:3,:].T, dst[0:3,:].T)

        # compute the transformation between the current source and 
        # nearest destination points
        T,_,_ = best_fit_transform(src[0:3,:].T, dst[0:3,indices].T)
        
        # For point-to-plane
#         distances, points = nearest_neighbor_planar(src[0:3,:].T, dst[0:3,:].T)
#         T,_,_ = best_fit_transform(src[0:3,:].T, points)

        # update the current source
        src = np.dot(T, src)

        # check error
        rms_error = np.sqrt(np.mean(np.square(distances)))
        print("RMS error:", rms_error)
        if abs(prev_error-rms_error) < threshold:
            break
        prev_error = rms_error

    # calculate final transformation
    T,_,_ = best_fit_transform(A, src[0:3,:].T)
    
    print("Number of iterations:", i+1)
    print("Time taken in seconds:", time.time() - start_time)

    return T, distances, i

In [8]:
df_1_clipped = df_1[df_1.z > -1.5]
df_2_clipped = df_2[df_2.z > -1.5]

A_clipped = df_1_clipped.iloc[:,0:3].as_matrix()
B_clipped = df_2_clipped.iloc[:,0:3].as_matrix()

output = icp(A_clipped, B_clipped, max_iterations=20, threshold=.001)

RMS error: 0.288025560862
RMS error: 0.201251053972
RMS error: 0.169700366323
RMS error: 0.161640321073
RMS error: 0.159725084073
RMS error: 0.159316324803
Number of iterations: 6
Time taken in seconds: 0.6679821014404297


In [9]:
A = df_1.iloc[:,0:3].as_matrix()
B = df_2.iloc[:,0:3].as_matrix()
T = output[0]

A_resized = np.vstack((A.T, np.ones((1, A.shape[0]))))
A_transformed = np.dot(T, A_resized).T[:,0:3]

# Variables for saving to .ply
src_final = A_transformed
dest_final = B
T

array([[  9.99999988e-01,   7.83963253e-05,   1.32662680e-04,
         -3.28567318e-01],
       [ -7.83889472e-05,   9.99999995e-01,  -5.56197240e-05,
          1.18680051e-02],
       [ -1.32667040e-04,   5.56093240e-05,   9.99999990e-01,
          4.10141034e-04],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00]])

### Combining Many CSV's Together

In [301]:
csv_list = []
csv_list.append('1503619201887427330-cloudpoint.csv')
csv_list.append('1503619202288320303-cloudpoint.csv')
csv_list.append('1503619202689218760-cloudpoint.csv')
csv_list.append('1503619203090113401-cloudpoint.csv')
csv_list.append('1503619203491182089-cloudpoint.csv')
csv_list.append('1503619203891901016-cloudpoint.csv')
csv_list.append('1503619203891901016-cloudpoint.csv')
csv_list.append('1503619204693687916-cloudpoint.csv')
csv_list.append('1503619205094586849-cloudpoint.csv')

for csv_filename in csv_list:
    df_1 = pd.read_csv(file_path + csv_filename, 
                 names = ['x', 'y', 'z', 
                          'intensity', 'ring', 'rotation', 'revolution'])
    df_1['distance'] = np.sqrt(df_1['x']**2 + df_1['y']**2 + df_1['z']**2)
    df_1 = df_1[df_1.distance > 2.5]
    df_1_clipped = df_1[df_1.z > -1.5]
    A_clipped = df_1_clipped.iloc[:,0:3].as_matrix()

    output = icp(A_clipped, B_clipped, max_iterations=60, threshold=.0001)

    A = df_1.iloc[:,0:3].as_matrix()
    B = df_2.iloc[:,0:3].as_matrix()
    T = output[0]
    A_resized = np.vstack((A.T, np.ones((1, A.shape[0]))))
    A_transformed = np.dot(T, A_resized).T[:,0:3]
    
    B = np.append(B, A_transformed, 0)
    
    df_2 = pd.DataFrame(data=B, columns=['x', 'y', 'z'])
    df_2['distance'] = np.sqrt(df_2['x']**2 + df_2['y']**2 + df_2['z']**2)
    df_2 = df_2[df_2.distance > 2.5]
    df_2_clipped = df_2[df_2.z > -1.5]
    B_clipped = df_2_clipped.iloc[:,0:3].as_matrix()

src_final = A_transformed
dest_final = B

RMS error: 1.92828460139
RMS error: 1.74618745328
RMS error: 1.58749801698
RMS error: 1.4337820908
RMS error: 1.28104324701
RMS error: 1.11836773257
RMS error: 0.965397275031
RMS error: 0.873765104526
RMS error: 0.829475293961
RMS error: 0.811526768646
RMS error: 0.803810716576
RMS error: 0.800245570955
RMS error: 0.798351835491
RMS error: 0.7974323982
RMS error: 0.797031355572
RMS error: 0.7968172042
RMS error: 0.796719048373
Number of iterations: 17
Time taken in seconds: 1.8311231136322021
RMS error: 2.81270129719
RMS error: 2.64800758759
RMS error: 2.47735081789
RMS error: 2.2909803391
RMS error: 2.11389328883
RMS error: 1.93324472691
RMS error: 1.77628777198
RMS error: 1.64806388814
RMS error: 1.53494341694
RMS error: 1.43249074126
RMS error: 1.33777824377
RMS error: 1.23770761413
RMS error: 1.13755133257
RMS error: 1.02312032827
RMS error: 0.886936224056
RMS error: 0.763193702476
RMS error: 0.689439132051
RMS error: 0.653196805766
RMS error: 0.636763657312
RMS error: 0.6288982213

RMS error: 3.53212801038
RMS error: 3.52923266157
RMS error: 3.52638971306
RMS error: 3.52303873673
RMS error: 3.5193016382
RMS error: 3.51337539066
RMS error: 3.50411965307
RMS error: 3.49462936714
RMS error: 3.48581145188
RMS error: 3.47762099321
RMS error: 3.46988465936
RMS error: 3.46481149378
RMS error: 3.46043915145
RMS error: 3.45599330081
RMS error: 3.45195958767
RMS error: 3.44864353574
RMS error: 3.44630268175
RMS error: 3.44449858538
RMS error: 3.44327200881
RMS error: 3.44264371237
RMS error: 3.44225585687
RMS error: 3.44189330528
RMS error: 3.44152917244
RMS error: 3.44118342729
RMS error: 3.44071320985
RMS error: 3.44020811207
RMS error: 3.43959802505
RMS error: 3.43898509431
RMS error: 3.43848316452
RMS error: 3.43809039333
RMS error: 3.43780783416
RMS error: 3.43763282115
Number of iterations: 60
Time taken in seconds: 26.39293909072876
RMS error: 2.08540144929
RMS error: 1.95488388229
RMS error: 1.86198960732
RMS error: 1.80278515552
RMS error: 1.75553958081
RMS error:

## Saving to .ply to Visualize Registration in Meshlab

In [302]:
src_ply_file_name = "src-pc-meshlab.ply"

with open(file_path + src_ply_file_name, 'w') as f:
    f.write("ply\n")
    f.write("format ascii 1.0\n")
    f.write("element vertex {}\n".format(src_final.shape[0]))
    f.write("property float32 x\n")
    f.write("property float32 y\n")
    f.write("property float32 z\n")
    f.write("end_header\n")
    
    for i in range(src_final.shape[0]):
        f.write("{} {} {}\n".format(src_final[i][0], 
                                    src_final[i][1], 
                                    src_final[i][2]))


dest_ply_file_name = "dest-pc-meshlab.ply"

with open(file_path + dest_ply_file_name, 'w') as f:
    f.write("ply\n")
    f.write("format ascii 1.0\n")
    f.write("element vertex {}\n".format(dest_final.shape[0]))
    f.write("property float32 x\n")
    f.write("property float32 y\n")
    f.write("property float32 z\n")
    f.write("end_header\n")
    
    for i in range(dest_final.shape[0]):
        f.write("{} {} {}\n".format(dest_final[i][0], 
                                    dest_final[i][1], 
                                    dest_final[i][2]))